In [1]:
import numpy as np
from tqdm import tqdm
import xarray as xr 
import matplotlib.pyplot as plt
import warnings
from itertools import product
from scipy.optimize import minimize
warnings.filterwarnings("ignore")
plt.style.use("plot_style.mplstyle")

In [2]:
p_bidi_dataset = xr.open_dataset("p_bidi_mpi_dataset_20240124.nc")

In [3]:
p_bidi_dataset

<xarray.Dataset>
Dimensions:                (mc: 101, fiber: 7, connector_return_loss: 1,
                            axis: 4, fiber_no: 7, bias_offset: 7,
                            tx_mzi_bias_offset: 7, tx_pa_bias_offset: 7,
                            rx_mzi_bias_offset: 7, rx_pa_bias_offset: 7,
                            ports: 4, bias: 4)
Coordinates:
  * mc                     (mc) float64 1.0 2.0 3.0 4.0 ... 99.0 100.0 101.0
  * fiber_no               (fiber_no) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0
  * bias_offset            (bias_offset) float64 -0.3 -0.2 -0.1 0.0 0.1 0.2 0.3
  * tx_mzi_bias_offset     (tx_mzi_bias_offset) float64 -0.3 -0.2 ... 0.2 0.3
  * tx_pa_bias_offset      (tx_pa_bias_offset) float64 -0.3 -0.2 ... 0.2 0.3
  * rx_mzi_bias_offset     (rx_mzi_bias_offset) float64 -0.3 -0.2 ... 0.2 0.3
  * rx_pa_bias_offset      (rx_pa_bias_offset) float64 -0.3 -0.2 ... 0.2 0.3
  * axis                   (axis) float64 1.0 2.0 3.0 4.0
  * ports                  (ports) float64 1.0 2.0 3.0 4.0
  * connector_return_loss  (connector_return_loss) int32 -30
Dimensions without coordinates: fiber, bias
Data variables:
    phase_shifts           (mc, fiber) float64 ...
    fiber_rotation         (fiber, mc, connector_return_loss, axis) float64 ...
    output_state_forward   (connector_return_loss, mc, ports, tx_pa_bias_offset, tx_mzi_bias_offset, rx_pa_bias_offset, rx_mzi_bias_offset) float64 ...
    output_state_reverse   (connector_return_loss, mc, ports, tx_pa_bias_offset, tx_mzi_bias_offset, rx_pa_bias_offset, rx_mzi_bias_offset) float64 ...
    optimization_bias      (connector_return_loss, mc, bias) float64 ...
    peak_bias              (connector_return_loss, mc, bias) float64 ...
Attributes:
    description:  Dataset for polarization controller MPI optimization
    author:       Ashwyn S
    created:      2024-01-24

In [4]:
p_bidi_sample = p_bidi_dataset.sel(mc=1, connector_return_loss = -30)

In [5]:
p_bidi_sample

<xarray.Dataset>
Dimensions:                (fiber: 7, axis: 4, fiber_no: 7, bias_offset: 7,
                            tx_mzi_bias_offset: 7, tx_pa_bias_offset: 7,
                            rx_mzi_bias_offset: 7, rx_pa_bias_offset: 7,
                            ports: 4, bias: 4)
Coordinates:
    mc                     float64 1.0
  * fiber_no               (fiber_no) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0
  * bias_offset            (bias_offset) float64 -0.3 -0.2 -0.1 0.0 0.1 0.2 0.3
  * tx_mzi_bias_offset     (tx_mzi_bias_offset) float64 -0.3 -0.2 ... 0.2 0.3
  * tx_pa_bias_offset      (tx_pa_bias_offset) float64 -0.3 -0.2 ... 0.2 0.3
  * rx_mzi_bias_offset     (rx_mzi_bias_offset) float64 -0.3 -0.2 ... 0.2 0.3
  * rx_pa_bias_offset      (rx_pa_bias_offset) float64 -0.3 -0.2 ... 0.2 0.3
  * axis                   (axis) float64 1.0 2.0 3.0 4.0
  * ports                  (ports) float64 1.0 2.0 3.0 4.0
    connector_return_loss  int32 -30
Dimensions without coordinates: fiber, bias
Data variables:
    phase_shifts           (fiber) float64 ...
    fiber_rotation         (fiber, axis) float64 ...
    output_state_forward   (ports, tx_pa_bias_offset, tx_mzi_bias_offset, rx_pa_bias_offset, rx_mzi_bias_offset) float64 ...
    output_state_reverse   (ports, tx_pa_bias_offset, tx_mzi_bias_offset, rx_pa_bias_offset, rx_mzi_bias_offset) float64 ...
    optimization_bias      (bias) float64 ...
    peak_bias              (bias) float64 ...
Attributes:
    description:  Dataset for polarization controller MPI optimization
    author:       Ashwyn S
    created:      2024-01-24

In [6]:
p_bidi_sample['output_state_forward'].variable

<xarray.Variable (ports: 4, tx_pa_bias_offset: 7, tx_mzi_bias_offset: 7,
                  rx_pa_bias_offset: 7, rx_mzi_bias_offset: 7)>
[9604 values with dtype=float64]

In [8]:
data_sample =  p_bidi_sample["output_state_forward"].sel(tx_pa_bias_offset=0.0, tx_mzi_bias_offset=0.0, rx_pa_bias_offset=0.0, rx_mzi_bias_offset=0.0).values

In [9]:
data_sample

array([1.81398498e-04, 4.47722775e-04, 7.34796685e-05, 9.65301372e-02])

In [10]:
data_sample =  p_bidi_sample["output_state_reverse"].sel(tx_pa_bias_offset=0.0, tx_mzi_bias_offset=0.0, rx_pa_bias_offset=0.0, rx_mzi_bias_offset=0.0).values

In [11]:
data_sample

array([0.00390454, 0.09696277, 0.00079803, 0.00010695])